<a href="https://colab.research.google.com/github/Uzmamushtaque/CSCI_4170_6170_Spring2026/blob/main/Lecture_10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lecture 10 — Natural Language Processing and Language Models

## Agenda
1. What is NLP? tasks, datasets, evaluation
2. Text preprocessing and tokenization
3. Vocabularies and embeddings
4. Language modeling (probabilities, objectives, metrics)
5. Seq2Seq + attention
6. Transformers and modern LLMs

## Learning objectives
By the end of this lecture you should be able to:
- Explain the standard NLP pipeline and where neural models fit.
- Compare word-, character-, and subword-tokenization and choose between them.
- Write the language-model objective using the chain rule and connect it to cross-entropy loss.
- Explain why full softmax is expensive and how sampled-softmax / NCE approximate it.
- Describe encoder–decoder (seq2seq) training vs inference, and why attention helps.
- Identify the key components of a Transformer (self-attention, multi-head, positional encoding, masking).


## 1) Introduction

**Natural Language Processing (NLP)** covers any computational method for working with human language (text or speech).  
Typical applications include translation, sentiment analysis, information extraction (NER), summarization, speech recognition, and text generation (chatbots / LLMs).

### Rule-based vs. ML-based vs. Foundation-model approaches
- **Rule-based / symbolic**: regex, grammars, dictionaries, deterministic pipelines.
- **Classical ML**: engineered features (n-grams, TF-IDF) + linear models (logistic regression, SVM).
- **Neural / deep learning**: learned representations (embeddings) + sequence models (RNN/LSTM/GRU, CNNs, Transformers).
- **Foundation models (LLMs)**: large Transformer models pretrained on broad text corpora; adapted via fine-tuning or prompting.

> Important: search engines and IR systems often rely on non-ML algorithms (indexing + retrieval) plus **optional** ML ranking models. Many “NLP products” are hybrid systems.

### Common NLP pipeline (high level)
1. **Collect data**: text corpus, labels (if supervised), train/val/test splits.
2. **Preprocess**: normalization (lowercasing, punctuation handling), tokenization, filtering.
3. **Represent text**: tokens → ids → embeddings.
4. **Model**: classifier / tagger / seq2seq model / language model.
5. **Train & evaluate**: metrics depend on task (accuracy/F1, BLEU/ROUGE, perplexity, etc.).
6. **Deploy**: latency, memory, safety constraints, monitoring for drift.

### Key NLP tasks (examples)
- **Tokenization**, **stemming** (reduce to a stem), **lemmatization** (map to dictionary form).
- **Part-of-speech tagging**, **named-entity recognition (NER)**.
- **Syntactic parsing** and **semantic analysis**.
- **Text classification** (sentiment, topic), **retrieval/ranking**, **generation**.

### Challenges in NLP
- Ambiguity and polysemy (same word, different meaning).
- Context dependence, pragmatics, sarcasm/irony.
- Domain shift (train on news, deploy on social media).
- Multilingual & low-resource settings.
- Long-range dependencies in sequences (motivation for attention/Transformers).


## 2) Vocabulary, tokens, and embeddings

Most language models and NLP pipelines start by converting raw text into **tokens**, then into **integer ids** from a **vocabulary**.

### Key terms
1. **Text corpus**: the collection of documents/sentences used for training/evaluation.
2. **Vocabulary**: the set of tokens the model can represent.
   - Usually includes **special tokens** such as: `[PAD]`, `[UNK]`/`OOV`, `[BOS]`/`<s>`, `[EOS]`/`</s>`.
   - Often built by frequency cutoff (keep top-*V* tokens) to control memory/compute.
3. **Tokenization**: mapping text → tokens. Common types:
   - **Word tokenization**: split into words (works well when word boundaries are clear).
   - **Character tokenization**: split into characters (handles OOV but sequences become long).
   - **Subword tokenization** (BPE, WordPiece, SentencePiece): split words into frequent pieces.
     - Example: “unbelievable” → `un`, `believ`, `able` (varies by tokenizer).
     - Helps with rare words and morphology; reduces OOV rate.
4. **Embeddings**: learned dense vectors for tokens.
   - A vocabulary of size *V* and embedding dimension *d* gives an embedding matrix **E ∈ R^{V×d}**.
   - Tokens become vectors via table lookup (much smaller than one-hot vectors).
   - Classic: Word2Vec, GloVe; modern: embeddings are usually learned jointly with the downstream model.

### Why subwords are dominant in modern LLMs
- Word vocabularies explode in size (names, typos, domain terms).
- Character models are robust but slower (longer sequences).
- Subwords are a strong trade-off: smaller vocab than words, shorter sequences than characters.

### Candidate sampling (why we care)
Training many NLP models (embeddings, language models) involves predicting a token among **V** possibilities.  
A naive softmax is **O(V)** per prediction step → expensive when V is large (e.g., 30k–200k+).

Approximations such as **sampled softmax**, **NCE**, or **negative sampling** reduce compute by only comparing against a small set of sampled “negative” tokens each step.


In [2]:
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences

# A tiny corpus (in practice you would have thousands/millions of sentences)
text_corpus = [
    "Bob ate apples, and pears.",
    "Fred ate apples!",
    "Bob did not eat bananas."
]

# Tokenizer maps tokens -> integer ids. oov_token catches unseen tokens at inference time.
tokenizer = tf.keras.preprocessing.text.Tokenizer(
    oov_token="[OOV]",
    filters=r'!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n'  # default filters; customize as needed
)

tokenizer.fit_on_texts(text_corpus)

print("Vocabulary size (including [OOV]):", len(tokenizer.word_index) + 1)  # +1 because index starts at 1
print("Word index (token -> id):")
print(tokenizer.word_index)

# Convert new texts to sequences of ids
new_texts = ["Bob ate fruit", "Fred ate pears", "Alice ate apples"]
seqs = tokenizer.texts_to_sequences(new_texts)
print("\nNew texts -> sequences:")
for t, s in zip(new_texts, seqs):
    print(f"{t!r} -> {s}")

# Padding: neural nets typically want fixed-length sequences
max_len = 6
padded = pad_sequences(seqs, maxlen=max_len, padding="post", truncating="post")
print("\nPadded sequences (post-padding to length", max_len, "):")
print(padded)

# Reverse lookup (id -> token) is useful for debugging
id_to_token = {idx: tok for tok, idx in tokenizer.word_index.items()}
id_to_token[0] = "[PAD]"
print("\nDecode the first padded sequence:")
print([id_to_token[i] for i in padded[0]])

Vocabulary size (including [OOV]): 14
Word index (token -> id):
{'[OOV]': 1, 'a': 2, 'bob': 3, 'e': 4, 'apples': 5, 'd': 6, 'pears': 7, 'fred': 8, 'did': 9, 'o': 10, 'ea': 11, 'ba': 12, 'as': 13}

New texts -> sequences:
'Bob ate fruit' -> [3, 2, 4, 1]
'Fred ate pears' -> [8, 2, 4, 7]
'Alice ate apples' -> [1, 2, 4, 5]

Padded sequences (post-padding to length 6 ):
[[3 2 4 1 0 0]
 [8 2 4 7 0 0]
 [1 2 4 5 0 0]]

Decode the first padded sequence:
['bob', 'a', 'e', '[OOV]', '[PAD]', '[PAD]']


## 3) Word probabilities and the language-model objective

A **language model (LM)** assigns a probability to a sequence of tokens:

$$
P(w_1, w_2, \dots, w_T)
$$

Using the **chain rule**:

$$
P(w_1, \dots, w_T) = \prod_{t=1}^{T} P(w_t \mid w_{<t})
$$

So the modeling problem becomes: **predict the next token** given the prefix (context).

### Training pairs (next-token prediction)
Example sentence:
`["She", "went", "to", "buy", "a", "book"]`

We can create input → target pairs such as:
- Input: `["She"]` → Target: `"went"`
- Input: `["She","went"]` → Target: `"to"`
- ...
- Input: `["She","went","to","buy","a"]` → Target: `"book"`

In practice we:
- Choose a **maximum context length** (sequence length) and create sliding windows.
- Use **padding** and **masking** so the model ignores `[PAD]` tokens.
- Train with teacher forcing (the ground-truth previous tokens are fed in during training).

### Evaluation metric: perplexity
Perplexity is a common LM metric:

$$
\text{PPL} = \exp\left(\frac{1}{N} \sum_{t=1}^{N} -\log P(w_t \mid w_{<t})\right)
$$

Lower perplexity ⇒ the model assigns higher probability to the true tokens.


### Mini-example: a simple n-gram language model

Before neural LMs, **n-gram models** approximated the context by only looking at the previous *(n−1)* tokens:

$$
P(w_t \mid w_{<t}) \approx P(w_t \mid w_{t-(n-1)}, \dots, w_{t-1})
$$

They are easy to implement, but struggle with:
- data sparsity (many rare contexts),
- long-range dependencies,
- generalization to unseen phrases (unless heavily smoothed).


In [3]:
# A tiny bigram LM (n=2) with add-k smoothing
from collections import Counter, defaultdict
import math

corpus = [
    "the cat sat on the mat",
    "the cat ate the fish",
    "the dog sat on the rug",
]

def tokenize(s):
    return ["<s>"] + s.lower().split() + ["</s>"]

bigrams = Counter()
unigrams = Counter()

for sent in corpus:
    toks = tokenize(sent)
    unigrams.update(toks)
    bigrams.update(zip(toks[:-1], toks[1:]))

V = len(unigrams)
k = 0.5  # smoothing strength

def p_bigram(w_prev, w):
    # add-k smoothing: (count(prev,w)+k)/(count(prev)+k*V)
    return (bigrams[(w_prev,w)] + k) / (unigrams[w_prev] + k*V)

test_sent = "the cat sat on the rug"
toks = tokenize(test_sent)

logp = 0.0
for a,b in zip(toks[:-1], toks[1:]):
    prob = p_bigram(a,b)
    logp += math.log(prob)
    print(f"P({b!r} | {a!r}) = {prob:.4f}")

ppl = math.exp(-logp / (len(toks)-1))
print("\nSentence:", test_sent)
print("Perplexity (bigram LM):", round(ppl, 3))


P('the' | '<s>') = 0.4118
P('cat' | 'the') = 0.2174
P('sat' | 'cat') = 0.2000
P('on' | 'sat') = 0.3333
P('the' | 'on') = 0.3333
P('rug' | 'the') = 0.1304
P('</s>' | 'rug') = 0.2308

Sentence: the cat sat on the rug
Perplexity (bigram LM): 4.011


## 4) Loss functions and scalable training

### Cross-entropy / negative log-likelihood (NLL)
For next-token prediction, the standard objective is to **maximize log-likelihood** (or minimize NLL):

$$
\mathcal{L} = -\sum_{t} \log P\big(w_t \mid w_{<t}\big)
$$

With a softmax over a vocabulary of size *V*:

$$
P(w_t=j \mid \cdot) = \frac{\exp(z_j)}{\sum_{i=1}^{V} \exp(z_i)}
$$

Computing the denominator is expensive when *V* is large.

### 1) Sampled softmax loss
Instead of normalizing over all *V* tokens, we sample **K negatives** and compute an approximate softmax.  
This reduces compute from **O(V)** to about **O(K)** per step (K ≪ V).

TensorFlow: `tf.nn.sampled_softmax_loss`  
https://www.tensorflow.org/api_docs/python/tf/nn/sampled_softmax_loss

### 2) Noise Contrastive Estimation (NCE)
Turns multiclass prediction into a **binary classification** problem: distinguish true data samples from noise samples.

TensorFlow: `tf.nn.nce_loss`  
https://www.tensorflow.org/api_docs/python/tf/nn/nce_loss

### Practical notes
- Candidate sampling works best when the sampled negatives are informative (not all ultra-rare).
- For small vocabularies, full softmax can be fine and simpler.
- In modern Transformer training, vocabulary sizes are often manageable (e.g., 30k–100k) but the sequence length and model size dominate compute.

Other important concepts when working with embeddings:
- **Cosine similarity**
- **Nearest neighbors / kNN** retrieval
- Analogy structure (classic Word2Vec behavior)


## 5) Language models (architectures)

A **language model** predicts the next token (or fills masked tokens) using a learned representation of context.

### Classic neural LMs
- **RNN**: maintains a hidden state that updates each time step.
- **LSTM / GRU**: gated RNNs that mitigate vanishing gradients, better for longer sequences.
- **CNN-based** sequence models (less common now for LMs).

### Transformer LMs
Transformers replace recurrence with **self-attention**, enabling:
- parallel training across tokens,
- better modeling of long-range dependencies (in practice),
- scaling to very large models/datasets.

### Output layer
Most LMs end with a projection from hidden state to vocabulary logits, then softmax (or approximations).

Useful reference:
- SOTA trackers for language modeling: https://paperswithcode.com/task/language-modelling


### Mini-example: train a tiny LSTM language model (toy)

This is **not** meant to be state-of-the-art—just to connect:
tokenization → dataset windows → Embedding/LSTM → softmax → perplexity.

On real corpora you would:
- use subword tokenization,
- larger models,
- regularization,
- GPUs/TPUs,
- careful train/val/test splits.


In [4]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers

toy_text = """we study natural language processing
language models predict the next token
attention helps models focus on relevant context
transformers scale efficiently
""".strip().splitlines()

# Build tokenization
tok = tf.keras.preprocessing.text.Tokenizer(oov_token='[OOV]')
tok.fit_on_texts(toy_text)
vocab_size = len(tok.word_index) + 1

# Create training sequences: predict next word from previous words (fixed window)
seq_len = 5  # context length
sequences = []
for line in toy_text:
    ids = tok.texts_to_sequences([line])[0]
    for i in range(1, len(ids)):
        start = max(0, i - seq_len)
        context = ids[start:i]
        target = ids[i]
        # left-pad context to fixed length
        context = [0]*(seq_len - len(context)) + context
        sequences.append((context, target))

X = np.array([c for c,_ in sequences], dtype=np.int32)
y = np.array([t for _,t in sequences], dtype=np.int32)

dataset = tf.data.Dataset.from_tensor_slices((X,y)).shuffle(256).batch(16)

# Define a simple LM
model = tf.keras.Sequential([
    layers.Embedding(vocab_size, 32, mask_zero=True),
    layers.LSTM(64),
    layers.Dense(vocab_size)  # logits
])

loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
model.compile(optimizer="adam", loss=loss_fn)

history = model.fit(dataset, epochs=10, verbose=0)
print("Final training loss:", history.history["loss"][-1])

# Compute perplexity on the training set (toy)
logits = model.predict(X, verbose=0)
nll = loss_fn(y, logits).numpy()
ppl = float(np.exp(nll))
print("Approx training perplexity:", round(ppl, 3))

# Greedy next-word generation from a prompt
id_to_word = {i:w for w,i in tok.word_index.items()}
id_to_word[0] = "[PAD]"

def generate(prompt, steps=8):
    context = tok.texts_to_sequences([prompt])[0]
    for _ in range(steps):
        ctx = context[-seq_len:]
        ctx = [0]*(seq_len-len(ctx)) + ctx
        pred = model.predict(np.array([ctx]), verbose=0)[0]
        next_id = int(np.argmax(pred))
        context.append(next_id)
    words = [id_to_word.get(i,'[OOV]') for i in context]
    return " ".join(words)

print("\nGenerated (greedy) from prompt 'attention helps':")
print(generate("attention helps", steps=8))


Final training loss: 3.021317481994629
Approx training perplexity: 20.464

Generated (greedy) from prompt 'attention helps':
attention helps models models token token token token token models


## 6) Seq2Seq (Encoder–Decoder) framework

**Sequence-to-sequence (seq2seq)** models map an input sequence to an output sequence:
- machine translation (English → French),
- summarization (document → summary),
- dialogue (user message → response).

### Training
1. **Encoder** consumes the input tokens and produces a representation (hidden states).
2. **Decoder** predicts the output tokens one-by-one, conditioned on the encoder output.

Important tokens:
- **SOS/BOS** (start-of-sequence)
- **EOS** (end-of-sequence)

### Teacher forcing
During training, the decoder typically receives the **ground-truth previous token** as input at each step (teacher forcing).  
At inference time, the decoder must use its **own** previously generated tokens.

### Inference strategies
- **Greedy decoding**: choose argmax at each step (fast, can be suboptimal).
- **Beam search**: keep top-*k* partial hypotheses (better quality, slower).


### Encoder/decoder states (LSTM/BiLSTM recap)

For an LSTM, the state is usually a pair:
- **h**: hidden state
- **c**: cell state

In TensorFlow/Keras, an LSTM layer can return:
- outputs over all time steps,
- final states `(h_T, c_T)`.

For a **BiLSTM**, you have:
- forward final states `(h_f, c_f)`,
- backward final states `(h_b, c_b)`.

To initialize a **unidirectional** decoder from a BiLSTM encoder, you need to **combine** these:
- concatenate: `h0 = [h_f ; h_b]` (then project if needed),
- or sum/average (less common),
- or use attention over all encoder outputs (common in modern seq2seq).

Key idea: the decoder needs a compact summary of the input **and/or** access to all encoder time-step outputs (attention).


## 7) Encoder–Decoder architecture (why it works)

The decoder uses the encoder’s representation so that generation is **conditioned** on the input.

### Single-layer intuition
- Encoder reads input sequence → final state summarizes input.
- Decoder starts from that final state → generates output sequence.

### Multi-layer case
If both encoder and decoder have multiple layers, it is common to:
- pass the final state from encoder layer *ℓ* to decoder layer *ℓ* as its initial state.

### Limitation
A single final state can be a bottleneck for long inputs.  
This motivates **attention**, which allows the decoder to consult *all* encoder states.


## 8) Attention mechanisms

Long-term dependencies are hard to capture using only the final encoder state.  
**Attention** allows the decoder to compute a **context vector** as a weighted combination of encoder outputs.

### Core idea (decoder attention over encoder outputs)
At decoder time step *t*:
1. Compute alignment scores between decoder state **s_t** and each encoder output **h_i**.
2. Normalize scores with softmax → attention weights **α_{t,i}**.
3. Compute context vector:
$$
c_t = \sum_i \alpha_{t,i} h_i
$$
4. Use **(s_t, c_t)** to predict the next token.

### Bahdanau vs. Luong (two classic variants)
- **Bahdanau attention** (additive): score uses a small neural net over `[s_t ; h_i]`.
- **Luong attention** (multiplicative): score uses dot product / bilinear form.

TensorFlow tutorial (NMT with attention):
https://www.tensorflow.org/text/tutorials/nmt_with_attention

Original attention paper (Bahdanau et al., 2014):
https://arxiv.org/pdf/1409.0473.pdf

### From encoder–decoder attention → self-attention
Transformers generalize attention by letting **every token attend to every other token** (self-attention), enabling rich contextual representations without recurrence.


## 9) Rise of Transformers and foundation models

A **foundation model** is typically a large Transformer pretrained on broad data.  
After pretraining, the same base model can be adapted to many tasks via:
- fine-tuning (supervised),
- instruction tuning,
- RLHF / preference optimization,
- or prompting (in-context learning).

Key properties that made Transformers dominant:
- parallelizable training (no recurrence),
- strong scaling behavior (more data/params → better performance),
- flexible architectures: encoder-only (BERT), decoder-only (GPT), encoder–decoder (T5).

Survey-style reference:
https://arxiv.org/abs/2108.07258


## 10) Transformers (high-level anatomy)

Transformers were introduced in *“Attention Is All You Need”* (Vaswani et al., 2017).  
https://arxiv.org/pdf/1706.03762.pdf

### Key components
1. **Token embeddings** + **positional encodings** (order information).
2. **Self-attention** (often multi-head):
   - each head attends with a different learned projection,
   - outputs are concatenated and projected back.
3. **Feed-forward network (FFN)** applied per token.
4. **Residual connections** + **layer normalization** (training stability).
5. **Masking** (for decoder-only / autoregressive models):
   - prevents attending to “future” tokens during training.

### Encoder-only vs decoder-only vs encoder–decoder
- **Encoder-only (BERT-like)**: bidirectional attention; good for understanding tasks (classification, NER).
- **Decoder-only (GPT-like)**: causal masked self-attention; good for generation.
- **Encoder–decoder (T5-like)**: encoder processes input, decoder generates output with cross-attention.

### Complexity note
Self-attention is **O(T²)** in sequence length *T*, so very long contexts require specialized techniques (sparse attention, memory, linear attention, etc.).


### Mini-example: Multi-Head Attention shapes (Transformer building block)

`tf.keras.layers.MultiHeadAttention` implements scaled dot-product attention with multiple heads.

In a decoder-only (GPT-like) model, we apply a **causal mask** so position *t* cannot attend to positions > *t*.


In [5]:
import tensorflow as tf
from tensorflow.keras import layers
import numpy as np

batch = 2
T = 6       # sequence length
d_model = 32
num_heads = 4

mha = layers.MultiHeadAttention(num_heads=num_heads, key_dim=d_model//num_heads)

# Dummy token representations (batch, time, d_model)
x = tf.random.normal((batch, T, d_model))

# Causal mask: allow attention to self and previous positions only
# mask shape: (batch, T, T) with 1 for allowed, 0 for blocked
causal = tf.linalg.band_part(tf.ones((T, T)), -1, 0)
causal = tf.cast(causal, tf.bool)
causal = tf.tile(causal[None, :, :], [batch, 1, 1])

y = mha(query=x, value=x, key=x, attention_mask=causal)
print("Input shape:", x.shape)
print("Output shape:", y.shape)

# You can also ask for attention scores (useful for visualization/debugging)
y2, scores = mha(query=x, value=x, key=x, attention_mask=causal, return_attention_scores=True)
print("Attention scores shape:", scores.shape)  # (batch, heads, T, T)


Input shape: (2, 6, 32)
Output shape: (2, 6, 32)
Attention scores shape: (2, 4, 6, 6)


## 11) Summary and quick checks

### Key takeaways
- NLP pipelines convert text → tokens → ids → embeddings → model.
- Language modeling uses the chain rule; training is cross-entropy / NLL.
- Full softmax is expensive for large vocabularies; sampled softmax and NCE are common approximations.
- Seq2seq models map input sequences to output sequences; attention helps overcome the encoder bottleneck.
- Transformers use self-attention + positional encodings and scale well; decoder-only Transformers use causal masking.

### Practice questions
1. Why does word-level tokenization lead to high OOV rates? How do subword tokenizers fix this?
2. Derive perplexity from the average negative log-likelihood.
3. What changes between seq2seq training (teacher forcing) and inference?
4. Compare encoder-only vs decoder-only Transformers: what tasks are they best at?
